In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/husein/t5/prepare/mesolitica-tpu.json'

In [2]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/speech/manglish/test-wav.tgz
# !tar -zxf test-wav.tgz
# !rm test-wav.tgz

In [3]:
from glob import glob

wavs = glob('test-wav/*.WAV')
len(wavs)

13189

In [4]:
import pandas as pd
df = pd.read_csv('test-wav/notes.csv')
df.head()

,wav_filename,wav_filesize,transcript
0,221930043.WAV,219052,b r sreenivasan ho see beng and sai hua kuan
1,221930091.WAV,172204,p c show kiki and kinohimitsu
2,221931702.WAV,149804,wantan mee is a traditional local cuisine
3,221930052.WAV,195276,antonia recently opened a new hainanese curry ...
4,221931492.WAV,155372,three two seven four nine six three zero


In [5]:
audios = df.values.tolist()

In [6]:
testset = []
for a in audios:
    testset.append((os.path.join('test-wav', a[0]), a[2]))

In [7]:
import tensorflow as tf
import malaya_speech
import json

subwords = malaya_speech.subword.load('transducer-singlish.subword')

def parse(serialized_example):

    data_fields = {
        'waveforms': tf.compat.v1.VarLenFeature(tf.float32),
        'targets': tf.compat.v1.VarLenFeature(tf.int64),
        'targets_length': tf.compat.v1.VarLenFeature(tf.int64),
    }
    features = tf.compat.v1.parse_single_example(
        serialized_example, features=data_fields
    )
    for k in features.keys():
        features[k] = features[k].values

    return features

def get_dataset(files, shuffle_size=32, num_cpu_threads=4,
                thread_count=24, is_training=False):
    def get():
        if is_training:
            d = tf.data.Dataset.from_tensor_slices(tf.constant(files))
            d = d.repeat()
            d = d.shuffle(buffer_size=len(files))
            cycle_length = min(num_cpu_threads, len(files))
            d = d.apply(
                tf.contrib.data.parallel_interleave(
                    tf.data.TFRecordDataset,
                    sloppy=is_training,
                    cycle_length=cycle_length))
            d = d.shuffle(buffer_size=100)
        else:
            d = tf.data.TFRecordDataset(files)
        d = d.map(parse, num_parallel_calls=thread_count)
        return d

    return get

In [8]:
with open('imda-tfrecords.json') as fopen:
    imda_tfrecord = json.load(fopen)
    
dev_dataset = get_dataset(imda_tfrecord['test'], is_training=False)()
features = dev_dataset.make_one_shot_iterator().get_next()
features

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


{'targets': <tf.Tensor 'IteratorGetNext:0' shape=(?,) dtype=int64>,
 'targets_length': <tf.Tensor 'IteratorGetNext:1' shape=(?,) dtype=int64>,
 'waveforms': <tf.Tensor 'IteratorGetNext:2' shape=(?,) dtype=float32>}

In [9]:
sess = tf.Session()

In [10]:
os.system('rm -rf testset-imda')
os.system('mkdir testset-imda')

0

In [11]:
import soundfile as sf
sf.write

<function soundfile.write(file, data, samplerate, subtype=None, endian=None, format=None, closefd=True)>

In [12]:
maxsize = 2000

index = 0
while True:
    try:
        r = sess.run(features)
        f = f'testset-imda/{index}.wav'
        sf.write(f, r['waveforms'], 16000)
        t = malaya_speech.subword.decode(subwords, r['targets'])
        testset.append((f, t))
        index += 1
        if index % 100 == 0:
            print(index)
        if index >= maxsize:
            break
    except Exception as e:
        print(e)
        break

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000


In [13]:
len(testset)

15189

In [16]:
with open('test-set-imda.json', 'w') as fopen:
    json.dump(testset, fopen)

In [17]:
!tar -zcf testset-imda.tar.gz testset-imda

In [18]:
b2_application_key_id = os.environ['b2_application_key_id']
b2_application_key = os.environ['b2_application_key']

In [19]:
from b2sdk.v1 import *
info = InMemoryAccountInfo()
b2_api = B2Api(info)
application_key_id = b2_application_key_id
application_key = b2_application_key
b2_api.authorize_account("production", application_key_id, application_key)
file_info = {'how': 'good-file'}
b2_bucket = b2_api.get_bucket_by_name('malay-dataset')

In [ ]:
outPutname = f'speech/manglish/testset-imda.tar.gz'
b2_bucket.upload_local_file(
    local_file='testset-imda.tar.gz',
    file_name=outPutname,
    file_infos=file_info,
)

In [ ]:
outPutname = f'speech/manglish/test-set-imda.json'
b2_bucket.upload_local_file(
    local_file='test-set-imda.json',
    file_name=outPutname,
    file_infos=file_info,
)